In [1]:
import ee
import geemap
import folium
import time

ee.Initialize()
Map = geemap.Map()

# Define the region of interest of Amazon basin
amazon_region = ee.Geometry.Polygon([[[-80.0, 10.0], 
                                      [-80.0, -10.0], 
                                      [-50.0, -10.0], 
                                      [-50.0, 10.0], 
                                      -80.0, 10.0]]]) 

print("START")

# Load GEDI Level 4A data
gedi1 =  ee.FeatureCollection('LARSE/GEDI/GEDI04_A_002/GEDI04_A_2020176035508_O08678_02_T04648_02_002_02_V002')\
        .filterBounds(amazon_region);
gedi2 =  ee.FeatureCollection('LARSE/GEDI/GEDI04_A_002/GEDI04_A_2020174070147_O08649_01_T04420_02_002_02_V002')\
        .filterBounds(amazon_region);

gedi = gedi1.merge(gedi2)
print('Number of filtered GEDI points:', gedi.size().getInfo())
# print('Number of GEDI points:', gedi.size().getInfo())
# Filter to keep only points with non-null 'agbd' values
gedi = gedi.filter(ee.Filter.notNull(['agbd']))
print('Number of filtered GEDI points:', gedi.size().getInfo())

# Map.setCenter(-60, 5, 5);
# Map.addLayer(merged_granules);

# load sentinel-1 data 
spring = ee.Filter.date('2022-03-01', '2022-04-20');
lateSpring = ee.Filter.date('2022-04-21', '2022-06-10');
summer = ee.Filter.date('2022-06-11', '2022-08-31');
# Define a masking function
def mask_edges(image):
    edge = image.lt(-30.0)  # Define an edge mask where values are less than -30
    masked_image = image.mask().And(edge.Not())  # Mask out edges
    return image.updateMask(masked_image)  # Apply the mask
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filterBounds(amazon_region)\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .filter(ee.Filter.inList('orbitProperties_pass', ['ASCENDING', 'DESCENDING']))
# Select the VV and VH bands
sentinel1_vv = sentinel1.select('VV')
sentinel1_vh = sentinel1.select('VH')
# Apply the masking function to each image in the collection
sentinel1_vv_masked = sentinel1_vv.map(mask_edges)
sentinel1_vv_final = ee.Image.cat(
        sentinel1_vv_masked.filter(spring).mean(),
        sentinel1_vv_masked.filter(lateSpring).mean(),
        sentinel1_vv_masked.filter(summer).mean());
# Apply the masking function to each image in the collection
sentinel1_vh_masked = sentinel1_vh.map(mask_edges)
sentinel1_vh_final = ee.Image.cat(
        sentinel1_vh_masked.filter(spring).mean(),
        sentinel1_vh_masked.filter(lateSpring).mean(),
        sentinel1_vh_masked.filter(summer).mean());


# Load Sentinel-2 surface reflectance data
# Sentinel-2 Level 2A multispectral imagery, ref:  https://www.sciencedirect.com/science/article/pii/S1569843222002965#s0010
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(amazon_region) \
                .filterDate('2021-01-01', '2021-12-31') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
# Define the bitmasks
cloud_bit_mask = ee.Number(1 << 5)  # Cloud bit is in the 6th bit position
cirrus_bit_mask = ee.Number(1 << 9)  # Cirrus bit is in the 10th bit position
# Apply the mask using bitwise AND to check that both cloud and cirrus bits are 0
def mask_clouds(image):
    qa = image.select('QA60')  # Select the QA60 band that holds cloud and cirrus bit information
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask)
sentinel2 = sentinel2.map(mask_clouds)
# Calculate NDVI
ndvi = sentinel2.map(lambda image: image.normalizedDifference(['B8', 'B4']).rename('NDVI')).median()
print('Number of sentinel2 points:', sentinel2.size().getInfo()) 
# Calculate EVI
def calculate_evi(image):
    return image.expression(
        '2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))',
        {
            'B8': image.select('B8'),
            'B4': image.select('B4'),
            'B2': image.select('B2')
        }).rename('EVI')
evi = sentinel2.map(calculate_evi).median()
print('Number of evi points:', evi.getInfo())


# Load Landsat 8 Surface Reflectance data
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
              .filterBounds(amazon_region) \
              .filterDate('2022-01-01', '2022-12-31') \
              .filter(ee.Filter.lt('CLOUD_COVER', 35))
# Calculate NDVI for Landsat 8
landsat_ndvi = landsat8.map(lambda image: image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')).median()
print('Number of landsat_ndvi points:', landsat_ndvi.getInfo())



# Load the GLO-30 DEM data from the COPERNICUS collection
dem = ee.ImageCollection('COPERNICUS/DEM/GLO30') \
          .filterBounds(amazon_region) \
          .mosaic()
# Calculate slope in degrees
slope = ee.Terrain.slope(dem)
# Calculate aspect in degrees
aspect = ee.Terrain.aspect(dem)
print('Number of dem points:')

# Stack all the features (Sentinel-1, Sentinel-2, Landsat, DEM)
feature_stack = sentinel1_vh_final.addBands(sentinel1_vv_final) \
                            .addBands(ndvi) \
                            .addBands(evi) \
                            .addBands(landsat_ndvi) \
                            .addBands(dem) \
                            .addBands(slope) \
                            .addBands(aspect)

print('Type of feature stack :', type(feature_stack))


# Sample the remote sensing data at GEDI footprint locations
training_data = feature_stack.sampleRegions(
    collection=gedi,
    properties=['agbd'],  # AGB values from GEDI
    scale=5000,
    tileScale=10
)

# print('Number of training_data points:', training_data.size().getInfo())
# print(training_data.first().getInfo())

# Train a Random Forest model
classifier = ee.Classifier.smileRandomForest(50).setOutputMode('REGRESSION')

print('Before trained_model points')
# Train the model
trained_model = classifier.train(
    features=training_data,
    classProperty='agbd',
    inputProperties=feature_stack.bandNames()
)

# Apply the trained model to predict AGB
agb_prediction = feature_stack.classify(trained_model)
print('Number of agb_prediction points:')
print(type(agb_prediction))


# # Export to Google Cloud Storage (optional)
# # task = ee.batch.Export.image.toCloudStorage({
# #     'image': agb_prediction,
# #     'description': 'AGB_Prediction_GCS',
# #     'bucket': 'test-project-agb',
# #     'fileNamePrefix': 'agb_prediction',
# #     'region': region,
# #     'scale': 100,
# #     'maxPixels': 1e13
# # })

# # task.start()

# # Assuming `agb_prediction` is an ee.Image and `region` is an ee.Geometry
# task = ee.batch.Export.image.toCloudStorage(
#     image=agb_prediction,         # Pass image directly, not in a dictionary
#     description='AGB_Prediction_GCS',  # Task description
#     bucket='test-agb-bucket',    # GCS bucket name
#     fileNamePrefix='agb_prediction',  # Prefix for the file name in the bucket
#     region=region,                # Region of interest (ee.Geometry)
#     scale=100,                    # Scale in meters
#     maxPixels=1e13,               # Max number of pixels
#     fileFormat='GeoTIFF'          # File format
# )

# # Start the task
# task.start()
# status = task.status()
# print(status)
# print("Export started")
# # Function to check task status periodically
# def check_task_status(task):
#     while True:
#         status = task.status()
#         state = status['state']
#         print('Current task state:', state)
        
#         if state == 'COMPLETED':
#             print("Export task completed successfully.")
#             break
#         elif state == 'FAILED':
#             print("Export task failed:", status)
#             break
        
#         # Wait before checking the status again
#         time.sleep(30)

# Start monitoring the task
# check_task_status(task)

# # # Define visualization parameters
# # # vis_params = {
# # #     'min': 0,
# # #     'max': 100,  # Adjust based on your data range
# # #     'palette': ['blue', 'green', 'yellow', 'orange', 'red']  # Color palette
# # # }

# # # # Create a folium map centered at a specific location (you can adjust this)
# # # map_center = [0, 0]  # Adjust to your area of interest
# # # my_map = folium.Map(location=map_center, zoom_start=10)

# # # # Add the Earth Engine image to the map
# # # agb_map_id = agb_prediction.getMapId(vis_params)
# # # folium.TileLayer(
# # #     tiles=agb_map_id['tile_fetcher'].url_format,
# # #     attr='Google Earth Engine',


# # #     overlay=True,
# # #     name='AGB Prediction'
# # # ).add_to(my_map)

# # # # Display the map
# # my_map
# print("End")

# # Model validation

# Split data into training and testing sets
# training_set = training_data.randomColumn().filter(ee.Filter.lt('random', 0.7))
# testing_set = training_data.randomColumn().filter(ee.Filter.gte('random', 0.7))

# # Evaluate the model on the testing set
# testing_classified = testing_set.classify(trained_model)
# error_matrix = testing_classified.errorMatrix('agbd', 'classification')

# # Print the RMSE and accuracy (optional)
# print('Error Matrix:', error_matrix.getInfo())

# Add the predicted AGB layer to the map D5C76C-Chinese Green, 
Map.addLayer(agb_prediction, 
  {'min': 0, 'max': 300, 'palette': ['5F9EA0', 'grey', 'yellow', '7CFC00','5F8575', '228B22','008000', '355E3B', '4F7942']}, 
  'Predicted AGB'
);
Map.centerObject(amazon_region, 6)
Map

START
Number of filtered GEDI points: 199250
Number of filtered GEDI points: 156643
Number of sentinel2 points: 26660
Number of evi points: {'type': 'Image', 'bands': [{'id': 'EVI', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Number of landsat_ndvi points: {'type': 'Image', 'bands': [{'id': 'NDVI', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Number of dem points:
Type of feature stack : <class 'ee.image.Image'>
Before trained_model points
Number of agb_prediction points:
<class 'ee.image.Image'>


KeyboardInterrupt: 